# 🎉 Welcome to the basics of working with genetic data! 🎉

---

###  **Let’s get started!** We will be going over some of the foundataional quality control measures and analyses in population genetics. Here’s a roadmap to guide you through the process:


### 🎯 **You’re off to a great start!**  
Let’s dive in!!  

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1iwJi7QfF3LElnCPen4fEQVoY4Vk4A9_5
!tar -xzvf TUTORIAL.tar.gz

In [126]:
import os
import gzip
from collections import defaultdict
import numpy as np
import pandas as pd
import allel
import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA as PCA
import pysam
from pyliftover import LiftOver
from typing import Dict, Generator, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import chi2
from scipy.stats import chi2 as chi2_dist
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster

#If you do not have a package here you can download via the command 'pip install ...' via your command line!

In [127]:
shared_qc_table = []
def count_variants(study_name, variant_data, sample_metadata, step_name):
    # Count variants by chromosome
    chroms = variant_data['variants/CHROM']
    autosomal = sum((chrom.isdigit() and 1 <= int(chrom) <= 22) for chrom in chroms)
    x_chr = sum(chrom in ['X', '23', '25'] for chrom in chroms)
    y_chr = sum(chrom in ['Y', '24'] for chrom in chroms)
    mt_chr = sum(chrom in ['MT', 'M', '26'] for chrom in chroms)
    
    # Count individuals by sex
    sex_codes = sample_metadata['sex']
    males = sum(sex == 1 for sex in sex_codes)
    females = sum(sex == 2 for sex in sex_codes)
    ambiguous = sum(sex == 0 for sex in sex_codes)
    individuals = len(sex_codes)
    
    # Add to shared QC table
    shared_qc_table.append([
        study_name,
        step_name,
        autosomal,
        x_chr,
        y_chr,
        mt_chr,
        individuals,
        males,
        females,
        ambiguous,
    ])
    
    return {
        "autosomal": autosomal,
        "x_chr": x_chr,
        "y_chr": y_chr,
        "mt_chr": mt_chr,
        "individuals": individuals,
        "males": males,
        "females": females,
        "ambiguous": ambiguous,
    }



# 🚀 LIFTOVER!  

## What is a Genome Build?  
A **genome build** is a **reference assembly** of a species' genome, providing a **standardized coordinate system** for genetic variants. Each build is an improved version of previous assemblies, correcting errors, adding missing sequences, and improving accuracy.  

### Human Genome Builds:  
- **hg18 (NCBI36)** – Released in **2006**  
- **hg19 (GRCh37)** – Released in **2009**  
- **hg38 (GRCh38)** – Released in **2013** (**Current Build**)  

---

## Why LiftOver?  
You wouldn’t use an outdated **operating system** on your computer, right?  
Similarly, we want to **update** our genomic data to the latest **genome build** to ensure accuracy.  

This process is called **"Lifting Over"** because we **Liftover** our old data to the newest build.  

### Failing to lift over can result in:  
❌ **Mismatched variant positions**  
❌ **Incorrect gene annotations**  
❌ **Data incompatibility** with current research tools  

---

## Let's Gather Our Tools!  

 

We are going to load out VCFs through SciKit-allele to convert it into a format that works within python!

In [128]:
def load_vcf(vcf_path: str) -> Tuple[Dict, Dict]:
    callset = allel.read_vcf(vcf_path)
    
    variant_data = {
        'CHROM': callset['variants/CHROM'],
        'POS': callset['variants/POS'],
        'ID': callset['variants/ID'],
        'REF': callset['variants/REF'],
        'ALT': callset['variants/ALT'][:, 0],
    }
    
    sample_metadata = {
        'samples': callset['samples'],
        'genotypes': callset['calldata/GT'],
        'sex': callset.get('samples/sex', np.zeros(len(callset['samples']), dtype=int))
    }
    
    return variant_data, sample_metadata


In [129]:
variant_data, sample_metadata = load_vcf("PATH")

# This will be a go to function we will be using to filter our data as we move along!

In [130]:
def filter_variants(variant_data: Dict, sample_metadata: Dict, max_missing: float = 0.05) -> Tuple[Dict, Dict]:

    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    missing_rate = gt.is_missing().mean(axis=1)
    keep_mask = missing_rate <= max_missing
    
    # Filter both variant data and genotypes
    filtered_variants = {
        k: v[keep_mask] for k, v in variant_data.items()
    }
    filtered_samples = {
        'samples': sample_metadata['samples'],
        'sex': sample_metadata['sex'],
        'genotypes': sample_metadata['genotypes'][keep_mask]
    }
    
    return filtered_variants, filtered_samples

## We are about to liftoff our liftOver! We will be using pyliftover which is a python package adapted from the University of Califorina: Santa Cruz's original liftOver function


In [131]:
def perform_liftover(variant_data: Dict, sample_metadata: Dict, chain_file: str) -> Tuple[Dict, Dict]:
    lo = LiftOver(chain_file)
    n_variants = len(variant_data['CHROM'])
    unmapped_mask = np.zeros(n_variants, dtype=bool)
    
    # Create copies to avoid modifying original data
    new_chroms = variant_data['CHROM'].copy()
    new_positions = variant_data['POS'].copy()
    
    for i, (chrom, pos) in enumerate(zip(variant_data['CHROM'], variant_data['POS'])):
        chrom_clean = chrom.replace('chr', '')
        lifted = lo.convert_coordinate(f'chr{chrom_clean}', pos - 1)  # Convert to 0-based
        
        if not lifted:
            unmapped_mask[i] = True
        else:
            # Update with lifted coordinates
            lifted_result = lifted[0]
            lifted_chrom = lifted_result[0]
            lifted_pos = lifted_result[1]
            new_chroms[i] = lifted_chrom
            new_positions[i] = lifted_pos + 1  # Convert back to 1-based
    
    # Update variant_data with new coordinates
    variant_data_lifted = variant_data.copy()
    variant_data_lifted['CHROM'] = new_chroms
    variant_data_lifted['POS'] = new_positions
    
    # Filter out unmapped variants
    return filter_variants(variant_data_lifted, sample_metadata, ~unmapped_mask)


In [132]:
variant_data, sample_metadata = perform_liftover(variant_data, sample_metadata, "PATH")

### We have lift off! Now lets really get this car into gear and update our data to reflect our new mapped variants!

# Ok! We have successfully performed a liftover! How do you feel? While the literal process of lifting over the coordinates has been completed, we are not quite done:
    
There’s still some housekeeping to take care of—after all, we didn’t generate this data ourselves, so we need to make sure everything’s in order. Here’s the game plan: 

---
- **Map to Reference FASTA**: 
    - Let’s make sure those lifted-over variants actually match the sequence of the new genome build.
- **Strand Flips**: 
    - Keep an eye out for major/minor (reference/alternate) allele mix-ups. Sometimes the reference strand orientation flips between builds, and we need to straighten that out.
- **Palindromic SNPs**: 
    - These variants are indistinguishable on forward and reverse strands, making them a headache for allele assignment. Since they’re too ambiguous to resolve confidently, we’ll filter them out entirely
- **Invalid SNPs**: 
    - Last but not least, filter out any variants where the reference allele doesn’t match the new build. Only the real deal makes the cut!
    

In [133]:
def align_to_reference(variant_data: Dict, sample_metadata: Dict, ref_fasta: str) -> Tuple[Dict, Dict]:
    ref = pysam.FastaFile(ref_fasta)
    n_variants = len(variant_data['CHROM'])
    
    # Validate dimensions match
    if sample_metadata['genotypes'].shape[0] != n_variants:
        raise ValueError(f"Dimension mismatch: {n_variants} variants vs {sample_metadata['genotypes'].shape[0]} genotypes")
    
    remove_mask = np.zeros(n_variants, dtype=bool)
    flip_mask = []
    force_ref_mask = []
    
    for i, (chrom, pos, var_id, a1, a2) in enumerate(zip(
        variant_data['CHROM'],
        variant_data['POS'],
        variant_data['ID'],
        variant_data['REF'],
        variant_data['ALT']
    )):
        # Skip if variant wasn't mapped
        if chrom is None or pos is None:
            remove_mask[i] = True
            continue
            
        # Handle chromosome naming
        chrom_fasta = chrom if chrom.startswith('chr') else f'chr{chrom}'
        if chrom_fasta not in ref.references:
            remove_mask[i] = True
            continue
            
        # Get reference base
        try:
            ref_base = ref.fetch(chrom_fasta, pos - 1, pos).upper()
        except:
            remove_mask[i] = True
            continue
            
        # Skip non-SNPs
        if a1 not in {'A', 'C', 'G', 'T'} or a2 not in {'A', 'C', 'G', 'T'}:
            remove_mask[i] = True
            continue
            
        # Skip palindromic SNPs
        if {a1, a2} in [{'A', 'T'}, {'C', 'G'}]:
            remove_mask[i] = True
            continue
            
        # Check allele alignment
        if ref_base == a2:
            force_ref_mask.append(i)
        elif ref_base != a1:
            # Check if flipped alleles match
            flip_map = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
            flipped_a1 = flip_map[a1]
            flipped_a2 = flip_map[a2]
            
            if ref_base == flipped_a2:
                force_ref_mask.append(i)
                flip_mask.append(i)
            elif ref_base == flipped_a1:
                flip_mask.append(i)
            else:
                remove_mask[i] = True
    
    # Apply allele flips
    for i in flip_mask:
        a1 = variant_data['REF'][i]
        a2 = variant_data['ALT'][i]
        variant_data['REF'][i] = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}[a1]
        variant_data['ALT'][i] = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}[a2]
    
    # Force reference allele
    for i in force_ref_mask:
        ref_allele = variant_data['REF'][i]
        alt_allele = variant_data['ALT'][i]
        variant_data['REF'][i] = alt_allele
        variant_data['ALT'][i] = ref_allele
    
    # Filter both variant data and genotypes
    keep_mask = ~remove_mask
    filtered_variants = {
        'CHROM': variant_data['CHROM'][keep_mask],
        'POS': variant_data['POS'][keep_mask],
        'ID': variant_data['ID'][keep_mask],
        'REF': variant_data['REF'][keep_mask],
        'ALT': variant_data['ALT'][keep_mask]
    }
    
    filtered_samples = {
        'samples': sample_metadata['samples'],
        'sex': sample_metadata['sex'],
        'genotypes': sample_metadata['genotypes'][keep_mask]
    }
    
    return filtered_variants, filtered_samples


In [134]:
variant_data, sample_metadata = align_to_reference(variant_data,  sample_metadata,"PATH")

# Now lets get to aligning! Be sure to remember where you sent the output of that lift over!

# Now we can feed that to our alignment!

---

Be sure that your studies are being sent to **different directories** to ensure there is not issues with the alignment process!

As you can see there was definetely some tidying to do, no matter how clean the data you inherited was!

#  We're Almost There! 

---

###  **Great progress so far!** We've tackled the bulk of the challenging work, and now we’re down to the final steps to ensure our data is polished and ready for standard quality control. Let’s break it down:

---

##  **Handling Duplicate Variants**  
Occasionally, multiple variants can appear at the same genomic position, which can complicate downstream analysis.  
**Our goal:**  
- Identify these duplicates and retain only the variant that is most prevalent in our dataset.   

---

##  **Standardizing Variant IDs**  
Different studies often use different genotyping chips, each with its own naming conventions for variant IDs. This inconsistency can create headaches when integrating multiple datasets.  

**Our solution:**  
- Rename variant IDs using a universal format: **`chr:pos:REF:ALT`**.  
- This ensures consistency across studies and makes our data more interoperable. Think of it as giving everyone the same map to follow! 🗺️  


In [135]:
def rename_variants(variant_data: Dict) -> Dict:
    """Rename variants using chr:pos:ref:alt format"""
    variant_data['ID'] = np.array([
        f"{chrom}:{pos}:{ref}:{alt}"
        for chrom, pos, ref, alt in zip(
            variant_data['CHROM'], variant_data['POS'],
            variant_data['REF'], variant_data['ALT']
        )
    ])
    return variant_data






In [136]:
variant_data = rename_variants(variant_data)

# Now that we have an updated and consistent dataset...time for analysis!!!

---
## **PCA**
# We are going to conduct a Principal Component Analysis (PCA) reduces genetic data to reveal the main axes of variation, showing how populations cluster and diverge based on genetic similarity.

---
## **GWAS**
# A Genome-Wide Association study (GWAS) is the bread and butter of genetic analysis, here we will be looking for variants statistically linked to traits or phenotypes tgat we may be interested in.

# Before we can get started, there is some additional QC that we need to do to make sure that the data is sound!

# We want to make sure that all the variants we look at are of relatively high genotyping in our dataset, so let us filter out variants that have exceptionally high missingness. 

In [137]:
def missingness(variant_data: Dict, sample_metadata: Dict, max_missing_rate: float = 0.05) -> Tuple[Dict, Dict]:
   
    # Convert genotypes to scikit-allel GenotypeArray
    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    
    # Calculate missingness
    missing = gt.is_missing()
    missing_per_variant = np.sum(missing, axis=1)  # Sum missing calls per variant
    total_samples = gt.n_samples
    missing_rate = missing_per_variant / total_samples
    
    # Create filter mask
    keep_mask = missing_rate <= max_missing_rate
    
    # Apply filtering to both variant data and genotypes
    filtered_variant_data = {
        'CHROM': variant_data['CHROM'][keep_mask],
        'POS': variant_data['POS'][keep_mask],
        'ID': variant_data['ID'][keep_mask],
        'REF': variant_data['REF'][keep_mask],
        'ALT': variant_data['ALT'][keep_mask]
    }
    
    filtered_sample_metadata = {
        'samples': sample_metadata['samples'],
        'sex': sample_metadata['sex'],
        'genotypes': sample_metadata['genotypes'][keep_mask]
    }
    
    return filtered_variant_data, filtered_sample_metadata


In [138]:
filtered_variants, filtered_samples = missingness(variant_data,sample_metadata,max_missing_rate=0.05)

# We’ll test if the variants in our dataset follow Hardy-Weinberg expectations. Deviations from HWE can indicate genotyping errors, population stratification, or other issues. It’s like a litmus test for data quality!  

In [139]:
def filter_by_hwe(variant_data: Dict,sample_metadata: Dict, p_threshold: float = 1e-25) -> Tuple[Dict, Dict]:

    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    n_variants = gt.shape[0]
    keep_mask = np.ones(n_variants, dtype=bool)
    
    for i in range(n_variants):
        # Get genotypes for this variant
        variant_genotypes = gt[i]
        
        # Count genotypes manually
        
        n_AA = np.sum((variant_genotypes == [0, 0]).all(axis=1))  # Homozygous reference
        n_Aa = np.sum(((variant_genotypes == [0, 1]).all(axis=1)) | 
                      ((variant_genotypes == [1, 0]).all(axis=1)))  # Heterozygous
        n_aa = np.sum((variant_genotypes == [1, 1]).all(axis=1))  # Homozygous alternate
        
        n = n_AA + n_Aa + n_aa
        
        if n == 0:
            keep_mask[i] = False
            continue
            
        # Calculate allele frequencies
        p = (2*n_AA + n_Aa) / (2*n)
        q = 1 - p
        
        # Skip if p or q is 0 (monomorphic)
        if p == 0 or q == 0:
            keep_mask[i] = False
            continue
            
        # Expected genotype frequencies
        exp_AA = p*p * n
        exp_Aa = 2*p*q * n
        exp_aa = q*q * n
        
        # Chi-square test 
        chi2 = 0
        if exp_AA > 0:
            chi2 += (n_AA - exp_AA)**2 / exp_AA
        if exp_Aa > 0:
            chi2 += (n_Aa - exp_Aa)**2 / exp_Aa
        if exp_aa > 0:
            chi2 += (n_aa - exp_aa)**2 / exp_aa
            
        # Convert to p-value
        
        pval = chi2_dist.sf(chi2, df=1)
        
        keep_mask[i] = pval >= p_threshold
    
    # Apply filtering
    filtered_variants = {k: v[keep_mask] for k, v in variant_data.items()}
    filtered_samples = {
        'samples': sample_metadata['samples'],
        'sex': sample_metadata['sex'],
        'genotypes': sample_metadata['genotypes'][keep_mask]
    }
    
    return filtered_variants, filtered_samples


In [140]:
variant_data, sample_metadata = filter_by_hwe(variant_data, sample_metadata)

# Filtering for minor allele frequency (MAF) trims out rare variants that are more likely to be sequencing noise or uninformative for analysis—like clearing static from a radio signal to hear the true genetic tune!

In [141]:
def filter_by_maf(variant_data: Dict, sample_metadata: Dict, maf_threshold: float = 0.01) -> Tuple[Dict, Dict]:

    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    allele_counts = gt.count_alleles()
    
    # Calculate MAF for each variant
    maf = np.min(allele_counts[:, :2] / np.sum(allele_counts[:, :2], axis=1)[:, np.newaxis], axis=1)
    
    # Create filter mask
    keep_mask = (maf >= maf_threshold) & ~np.isnan(maf)
    
    # Apply filtering
    filtered_variants = {
        k: v[keep_mask] for k, v in variant_data.items()
    }
    filtered_samples = {
        'samples': sample_metadata['samples'],
        'sex': sample_metadata['sex'],
        'genotypes': sample_metadata['genotypes'][keep_mask]
    }
    
    return filtered_variants, filtered_samples



def run_pca(variant_data: Dict, sample_metadata: Dict, population_file: str, n_components: int = 10):

    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    pop_df = pd.read_csv(population_file, sep='\t',  header=None, names=['sample_id', 'population'])
    pop_dict = dict(zip(pop_df['sample_id'], pop_df['population']))
    # Convert genotypes to allele counts (0,1,2) and take every 5th variant
    gn = gt.to_n_alt()[::5]  

    # Transpose to get samples as rows (n_samples × n_variants)
    gn_t = gn.T
    
    # Standardize the data 
    scaler = sklearn.preprocessing.StandardScaler()
    gn_std = scaler.fit_transform(gn_t)
    
    # Perform PCA
    pca = sklearn.decomposition.PCA(n_components=n_components)
    pcs = pca.fit_transform(gn_std)
    sample_ids = sample_metadata['samples']
    populations = [pop_dict.get(sample_id, 'Unknown') for sample_id in sample_ids]
    unique_pops = list(set(populations))
    colors = ["#4363d8","#e6194b","#3cb44b","#f58231","#911eb4","#33b2b2","#af2ca8","#96c603","#ca5d5d","#008080", "#654977","#29699E","#8f862d","#800000","#4f9c66","#808000","#6e75b7","#000075","#662245","#ff0000", "#DCD42E", "#44af69", "#f8333c", "#fcab10", "#2b9eb3", "#dbd5b5", "#ffdd19"]
    color_map = dict(zip(unique_pops, colors))
    
    # Plot first two PCs
    plt.figure(figsize=(10, 8))
    for pop in unique_pops:
        mask = [p == pop for p in populations]
        plt.scatter(pcs[mask, 0], pcs[mask, 1], 
                   c=[color_map[pop]], 
                   label=pop, 
                   alpha=1, 
                   s=70)
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    
    return pcs




In [142]:
# Perform MAF filtering
maf_filtered_variants, maf_filtered_samples = filter_by_maf(variant_data, sample_metadata)

# And now we can finally run the PCA! Keep your eye out for distinctive clusters! 

In [143]:
# Run PCA 
pcs = run_pca(maf_filtered_variants, maf_filtered_samples, "PATH", n_components=10)


# Now we can do the GWAS, we will be visuallizing the results of our GWAS in a plot known as a Manhattan Plot; A Manhattan plot shows GWAS results as a skyline of p-values.  

In [144]:
def association_linregress(genotypes, phenotypes):
    p_values = []
    for SNP_index in range(genotypes.shape[0]): 
        x= genotypes[SNP_index, :]
        if np.all(x == x[0]):
            p_values.append(np.nan)
            continue
        results= stats.linregress(x,phenotypes)
        p_values.append(results.pvalue) 
    
    return p_values
def run_gwas(variant_data: Dict, sample_metadata: Dict, phenotype_file: str):
  
    # Phenotype data 
    pheno_df = pd.read_csv(phenotype_file, sep='\t',  header=None, names=['sample', 'phenotype'])
    pheno_df = pheno_df.set_index('sample')
    
    #  Align genotypes with phenotypes
    sample_ids = sample_metadata['samples']
    phenotypes = pheno_df.loc[sample_ids]['phenotype'].values
    
    #  Convert genotypes to allele counts (0,1,2)
    gt = allel.GenotypeArray(sample_metadata['genotypes'])
    gn = gt.to_n_alt()  # shape: (n_variants, n_samples)
    
    #  Run regression
    p_values = association_linregress(gn, phenotypes)
    
    # Create Manhattan plot
    plt.figure(figsize=(15, 5))
    
    # Prepare data for plotting
    chrom = variant_data['CHROM']
    pos = variant_data['POS']
    
    
    # Convert chromosome names to numeric
    unique_chroms = set(chrom)
    def parse_chromosome_name(chrom_name):
        chrom_part = chrom_name.replace('chr', '')  # Remove 'chr' prefix
        main_part = chrom_part.split('_')[0]       # Some datasets with annotate their chromosomes with underscores!
    
        if main_part.isdigit():
            return int(main_part)   
        elif main_part == 'X':
            return 23               
        elif main_part == 'Y':
            return 24  
        elif main_part == 'MT' or main_part == 'M':
            return 25  
        else:
            return 0 
    chrom_num = [parse_chromosome_name(c) for c in chrom]
    # Plotting Dataframe
    gwas_df = pd.DataFrame({'CHR': chrom_num,'BP': pos,'P': p_values})
    
    # Manhattan plot 
    gwas_df = gwas_df.sort_values(['CHR', 'BP'])
    gwas_df['ind'] = range(len(gwas_df))
    gwas_df_grouped = gwas_df.groupby('CHR')
    
    colors = ['#fc0f0f', '#fc9a0f','#fcf80f', '#4efc0f', '#0fa5fc', '#c10ffc','#fc0fb5']  
    x_labels = []
    x_labels_pos = []
    
    for num, (name, group) in enumerate(gwas_df_grouped):
        color = colors[num % len(colors)]
        plt.scatter(group['ind'], -np.log10(group['P']), 
                   color=color, s=2)
        x_labels.append(name)
        x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
    
    plt.axhline(y=-np.log10(5e-8), color='r', linestyle='--')  # Genome-wide significance
    plt.xticks(x_labels_pos, x_labels)
    plt.xlabel('Chromosome')
    plt.ylabel('-log10(p-value)')
    plt.title('Manhattan Plot')
    plt.tight_layout()
    plt.show()
    
    return p_values, gwas_df


In [145]:
p_values, gwas_results = run_gwas(maf_filtered_variants, maf_filtered_samples,"PATH")

# Can you guess what the phenotype was based on the results?

# Now we can save our work! This will likely take several minutes to run!

In [146]:

def save_vcf(variant_data: dict, sample_metadata: dict, output_path: str):
    """
    Fast VCF writer using manual construction for speed.
    """
    samples = sample_metadata['samples']
    genotypes = sample_metadata['genotypes']  # shape: (num_variants, num_samples, 2)

    with gzip.open(output_path, 'wt') as f:
        # Write VCF header
        f.write("##fileformat=VCFv4.2\n")
        f.write("##source=Generated by liftover pipeline\n")
        for chrom in sorted(set(variant_data['CHROM'])):
            chrom_str = f"chr{chrom}" if not str(chrom).startswith("chr") else str(chrom)
            f.write(f"##contig=<ID={chrom_str}>\n")
        f.write("##FORMAT=<ID=GT,Number=1,Type=String,Description=\"Genotype\">\n")
        f.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t")
        f.write("\t".join(samples) + "\n")

        # Write variants
        num_variants = len(variant_data['CHROM'])
        for i in range(num_variants):
            chrom = variant_data['CHROM'][i]
            chrom_str = f"chr{chrom}" if not str(chrom).startswith("chr") else str(chrom)
            pos = variant_data['POS'][i]
            vid = variant_data['ID'][i]
            ref = variant_data['REF'][i]
            alt = variant_data['ALT'][i] if variant_data['ALT'][i] not in ('.', None) else '.'
            qual = str(variant_data['QUAL'][i]) if 'QUAL' in variant_data else '.'
            filt = variant_data['FILTER'][i] if 'FILTER' in variant_data else '.'
            if filt in (None, '.'):
                filt = '.'
            info = '.'
            fmt = 'GT'

            # Process genotypes as strings
            gt_strings = []
            for j in range(len(samples)):
                gt = genotypes[i, j]
                if gt[0] == -1 or gt[1] == -1:
                    gt_str = './.'
                else:
                    gt_str = f"{int(gt[0])}/{int(gt[1])}"
                gt_strings.append(gt_str)

            line = f"{chrom_str}\t{pos}\t{vid}\t{ref}\t{alt}\t{qual}\t{filt}\t{info}\t{fmt}\t" + "\t".join(gt_strings)
            f.write(line + "\n")

save_vcf(variant_data, sample_metadata, "PATH")

# 🎉 **You Did It! Look at You Go, Hot Shot!** 🎉
